#### Introduction to Statistical Learning, Exercise 3.5

__Please do yourself a favour and only look at the solutions after you honestly tried to solve the exercises.__

# The Collinearity Problem

We will simulate a data set that has high collinearity among predictors and investigate it with the help of our knowledge of the simulated truth.

We will use `numpy`'s random generator facilities for the simulation. If you want reproducible results you should set the random seed explicitly like this at the beginning (the actual seed value does not matter):

```python
np.random.seed(seed=123)
```

 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from islpy import datasets, utils, lmplots
sns.set()
%matplotlib inline